In [1]:
from multifora_api import *
import pandas as pd
import numpy as  np

In [1]:
import os
from unittest import TestCase

In [2]:
from werkzeug.exceptions import NotFound

In [3]:
class TestMlApi(TestCase):
    def test_length():
        a = ['LogisticRegression',
 'RandomForestClassifier',
 'LinearRegression',
 'RandomForestRegressor',
 'DecisionTreeClassifier',
 'DecisionTreeRegressor',
 'KNeighborsClassifier',
 'KNeighborsRegressor']

In [4]:
f=TestMlApi()

In [6]:
# загружаем train dataset
from multifora_api import *
import pandas as pd
import numpy as  np
data = pd.read_csv('train.csv')

In [2]:
# загружаем train dataset
data = pd.read_csv('train.csv')

In [3]:
data['target'] = 1
data.loc[data['store_and_fwd_flag'] == 'N', 'target'] = 0

In [4]:
# каждый инстанс модели имеет уникальный uuid сессии. Каждый новый инстанс имеет доступ к личном простарнству для 
# хранения выборок и результатов работы моделей
api = MlApi(host='http://<server ip>:<port>/')

InvalidURL: Failed to parse: http://<server ip>:<port>//api/login

In [5]:
# загружаем train dataset на сервер
api.load_dataset('train',data.sample(5000))

data uploading...
data upload


In [6]:
# добавляем модели 
api.add_model('LG','LogisticRegression',{})
api.add_model('RF','RandomForestClassifier',{})

LG added
RF added


In [7]:
api.gel_model_list()

,Model name,fitted,params,type
0,LG,False,{},LogisticRegression
1,RF,False,{},RandomForestClassifier


In [8]:
# так можно узнать список доступных классов моделей на сервере
api.models_available()

['LogisticRegression',
 'RandomForestClassifier',
 'LinearRegression',
 'RandomForestRegressor',
 'DecisionTreeClassifier',
 'DecisionTreeRegressor',
 'KNeighborsClassifier',
 'KNeighborsRegressor']

In [9]:
# обчаем модель на факторах из short_list с целевой переменной target
short_list = ['passenger_count', 'pickup_longitude', 'pickup_latitude']
api.fit_model('LG', short_list, 'target')

Fitted!


In [10]:
api.gel_model_list()

,Model name,fitted,params,type
0,LG,True,{},LogisticRegression
1,RF,False,{},RandomForestClassifier


In [11]:
# подгружаем eval dataset
# data = pd.read_csv('set_test.csv')
# data['target'] = 1
# data.loc[data['store_and_fwd_flag'] == 'N', 'target'] = 0

api.load_dataset('eval',data.sample(5000))

data uploading...
data upload


In [12]:
# оценим модель по мтерике accuracy и roc_auc. Для того, чтобы задать другую метрику необходимо
# в словаре metrics в качестве ключа дать название класса из https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics
# а в качестве значения, что необходимо метрике: предсказание моделей(predict) или вывод вероятностей модели(predict_proba)
metrics = {
    'accuracy_score':'predict',
    'roc_auc_score' : 'predict_proba'
}
api.evaluate_model('LG', metrics)

Evaluating LG...
Evaluated!


In [13]:
# выводим реузльтаты оценки модели
api.get_evaluation_results()

{'LG': {'accuracy_score': 0.994, 'roc_auc_score': 0.6235949027498323},
 'RF': {}}

In [14]:
# так можно получить предсказания для любого сета (параметр функции sample_type)
preds = api.predict_model('LG', 'predict_proba', 'eval')

In [15]:
# так можно удалить модель
api.delete_model('LG')

Model LG deleted


In [16]:
api.gel_model_list()

,Model name,fitted,params,type
0,RF,False,{},RandomForestClassifier


In [17]:
api.fit_model('RF', short_list, 'target')

Fitted!


In [18]:
# давайте дадим новые гиперпарметры для RF 
# (!!очень важно!!: если изменить гиперпараметры, то информация об оценки модели сотрется, а также необходимо будет заного её обучить)
api.change_hyperparams('RF', params = {'n_estimators':1000, 'random_state' : 42})

Params updated!


In [19]:
api.gel_model_list()

,Model name,fitted,params,type
0,RF,False,"{'n_estimators': 1000, 'random_state': 42}",RandomForestClassifier


In [20]:
# по понятным причинам у нас не получилось оценить модель, так как она еще не обучена
metrics = {
    'accuracy_score':'predict',
    'roc_auc_score' : 'predict_proba'
}
api.evaluate_model('RF', metrics)

Evaluating RF...


Exception: Model not fitted

In [22]:
# Мы получили некий task id ниже. Все операции данного api выполняются отдельным приложением. 
# Если данный task не успел выполниться за секунду, то мы получаем сообщение ниже. 
# Необходимо проверить результат позднее. Для проверки используйте функцию result(). Пример в ячейке ниже.
api.fit_model('RF', short_list, 'target')

Your task in progress. Task id = 476db973-509d-4c82-a234-4b570f2750b3. Check result later. For checking use result() function.


SystemExit: 0

In [23]:
api.result('476db973-509d-4c82-a234-4b570f2750b3')

'Fitted!'

In [24]:
metrics = {
    'accuracy_score':'predict',
    'roc_auc_score' : 'predict_proba'
}
api.evaluate_model('RF', metrics)

Evaluating RF...
Your task in progress. Task id = 1f20d95b-4c33-49f6-8519-26b0c58a37f6. Check result later. For checking use result() function.


SystemExit: 0

In [25]:
api.result('1f20d95b-4c33-49f6-8519-26b0c58a37f6')

'Evaluated!'

In [26]:
api.get_evaluation_results()

{'RF': {'accuracy_score': 0.9948, 'roc_auc_score': 0.44993468649517676}}